In [1]:
import redis
import time
import json

In [2]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)
p = r.pubsub(ignore_subscribe_messages=True)

In [3]:
def meet_puck_linear_y(y_len, bot_state, puck_state):
    if (y_len >= 40):
        new_bot = bot_state
        
        new_bot['y'] = bot_state['y'] - 10
        
        r.set("machine-state-bot", json.dumps(new_bot))
        r.publish('state-changed', True)
    else:
        print 'Should be met...'  

In [4]:
def meet_puck_linear_x(x_len, bot_state):
    print 'meet puck linear x'

In [5]:
def meet_the_puck(puck_state, bot_state):
    #print 'Puck State: ' + puck_state + ' Bot State: ' + bot_state
    p = json.loads(puck_state)
    b = json.loads(bot_state)

    px = int(p['x'])
    py = int(p['y'])
    
    print "px", px
    print "py", py
    
    bx = int(b['x'])
    by = int(b['y'])
    
    print "bx", bx
    print "by", by
        
    y_len = abs(py - by)
    print 'y_len', y_len
    
    x_len = abs(px - bx)
    print 'x_len', x_len
    
    # if x <= 0, move linear towards puck
    if x_len <= 0:
        meet_puck_linear_y(y_len, b, p)
    else:
        print 'Move at angle'

In [6]:
p.subscribe('state-changed')

In [ ]:
# event loop for bot
while True:
    message = p.get_message()
    if message:
        if message['channel'] == 'state-changed':
            puck_state = r.get('machine-state-puck')
            bot_state = r.get('machine-state-bot')
            meet_the_puck(puck_state, bot_state)
        time.sleep(0.001)

px 240
py 50
bx 240
by 590
y_len 540
x_len 0
px 240
py 50
bx 240
by 580
y_len 530
x_len 0
px 240
py 50
bx 240
by 570
y_len 520
x_len 0
px 240
py 50
bx 240
by 560
y_len 510
x_len 0
px 240
py 50
bx 240
by 550
y_len 500
x_len 0
px 240
py 50
bx 240
by 540
y_len 490
x_len 0
px 240
py 50
bx 240
by 530
y_len 480
x_len 0
px 240
py 50
bx 240
by 520
y_len 470
x_len 0
px 240
py 50
bx 240
by 510
y_len 460
x_len 0
px 240
py 50
bx 240
by 500
y_len 450
x_len 0
px 240
py 50
bx 240
by 490
y_len 440
x_len 0
px 240
py 50
bx 240
by 480
y_len 430
x_len 0
px 240
py 50
bx 240
by 470
y_len 420
x_len 0
px 240
py 50
bx 240
by 460
y_len 410
x_len 0
px 240
py 50
bx 240
by 450
y_len 400
x_len 0
px 240
py 50
bx 240
by 440
y_len 390
x_len 0
px 240
py 50
bx 240
by 430
y_len 380
x_len 0
px 240
py 50
bx 240
by 420
y_len 370
x_len 0
px 240
py 50
bx 240
by 410
y_len 360
x_len 0
px 240
py 50
bx 240
by 400
y_len 350
x_len 0
px 240
py 50
bx 240
by 390
y_len 340
x_len 0
px 240
py 50
bx 240
by 380
y_len 330
x_len 0
px 240
py 

px 240
py 50
bx 240
by 590
y_len 540
x_len 0
px 240
py 50
bx 240
by 580
y_len 530
x_len 0
px 240
py 50
bx 240
by 570
y_len 520
x_len 0
px 240
py 50
bx 240
by 560
y_len 510
x_len 0
px 240
py 50
bx 240
by 550
y_len 500
x_len 0
px 240
py 50
bx 240
by 540
y_len 490
x_len 0
px 240
py 50
bx 240
by 530
y_len 480
x_len 0
px 240
py 50
bx 240
by 520
y_len 470
x_len 0
px 240
py 50
bx 240
by 510
y_len 460
x_len 0
px 240
py 50
bx 240
by 500
y_len 450
x_len 0
px 240
py 50
bx 240
by 490
y_len 440
x_len 0
px 240
py 50
bx 240
by 480
y_len 430
x_len 0
px 240
py 50
bx 240
by 470
y_len 420
x_len 0
px 240
py 50
bx 240
by 460
y_len 410
x_len 0
px 240
py 50
bx 240
by 450
y_len 400
x_len 0
px 240
py 50
bx 240
by 440
y_len 390
x_len 0
px 240
py 50
bx 240
by 430
y_len 380
x_len 0
px 240
py 50
bx 240
by 420
y_len 370
x_len 0
px 240
py 50
bx 240
by 410
y_len 360
x_len 0
px 240
py 50
bx 240
by 400
y_len 350
x_len 0
px 240
py 50
bx 240
by 390
y_len 340
x_len 0
px 240
py 50
bx 240
by 380
y_len 330
x_len 0
px 240
py 

px 240
py 50
bx 240
by 590
y_len 540
x_len 0
px 240
py 50
bx 240
by 580
y_len 530
x_len 0
px 240
py 50
bx 240
by 570
y_len 520
x_len 0
px 240
py 50
bx 240
by 560
y_len 510
x_len 0
px 240
py 50
bx 240
by 550
y_len 500
x_len 0
px 240
py 50
bx 240
by 540
y_len 490
x_len 0
px 240
py 50
bx 240
by 530
y_len 480
x_len 0
px 240
py 50
bx 240
by 520
y_len 470
x_len 0
px 240
py 50
bx 240
by 510
y_len 460
x_len 0
px 240
py 50
bx 240
by 500
y_len 450
x_len 0
px 240
py 50
bx 240
by 490
y_len 440
x_len 0
px 240
py 50
bx 240
by 480
y_len 430
x_len 0
px 240
py 50
bx 240
by 470
y_len 420
x_len 0
px 240
py 50
bx 240
by 460
y_len 410
x_len 0
px 240
py 50
bx 240
by 450
y_len 400
x_len 0
px 240
py 50
bx 240
by 440
y_len 390
x_len 0
px 240
py 50
bx 240
by 430
y_len 380
x_len 0
px 240
py 50
bx 240
by 420
y_len 370
x_len 0
px 240
py 50
bx 240
by 410
y_len 360
x_len 0
px 240
py 50
bx 240
by 400
y_len 350
x_len 0
px 240
py 50
bx 240
by 390
y_len 340
x_len 0
px 240
py 50
bx 240
by 380
y_len 330
x_len 0
px 240
py 

px 240
py 50
bx 240
by 590
y_len 540
x_len 0
px 240
py 50
bx 240
by 580
y_len 530
x_len 0
px 240
py 50
bx 240
by 570
y_len 520
x_len 0
px 240
py 50
bx 240
by 560
y_len 510
x_len 0
px 240
py 50
bx 240
by 550
y_len 500
x_len 0
px 240
py 50
bx 240
by 540
y_len 490
x_len 0
px 240
py 50
bx 240
by 530
y_len 480
x_len 0
px 240
py 50
bx 240
by 520
y_len 470
x_len 0
px 240
py 50
bx 240
by 510
y_len 460
x_len 0
px 240
py 50
bx 240
by 500
y_len 450
x_len 0
px 240
py 50
bx 240
by 490
y_len 440
x_len 0
px 240
py 50
bx 240
by 480
y_len 430
x_len 0
px 240
py 50
bx 240
by 470
y_len 420
x_len 0
px 240
py 50
bx 240
by 460
y_len 410
x_len 0
px 240
py 50
bx 240
by 450
y_len 400
x_len 0
px 240
py 50
bx 240
by 440
y_len 390
x_len 0
px 240
py 50
bx 240
by 430
y_len 380
x_len 0
px 240
py 50
bx 240
by 420
y_len 370
x_len 0
px 240
py 50
bx 240
by 410
y_len 360
x_len 0
px 240
py 50
bx 240
by 400
y_len 350
x_len 0
px 240
py 50
bx 240
by 390
y_len 340
x_len 0
px 240
py 50
bx 240
by 380
y_len 330
x_len 0
px 240
py 

px 240
py 50
bx 240
by 520
y_len 470
x_len 0
px 240
py 50
bx 240
by 510
y_len 460
x_len 0
px 240
py 50
bx 240
by 500
y_len 450
x_len 0
px 240
py 50
bx 240
by 490
y_len 440
x_len 0
px 240
py 50
bx 240
by 480
y_len 430
x_len 0
px 240
py 50
bx 240
by 470
y_len 420
x_len 0
px 240
py 50
bx 240
by 460
y_len 410
x_len 0
px 240
py 50
bx 240
by 450
y_len 400
x_len 0
px 240
py 50
bx 240
by 440
y_len 390
x_len 0
px 240
py 50
bx 240
by 430
y_len 380
x_len 0
px 240
py 50
bx 240
by 420
y_len 370
x_len 0
px 240
py 50
bx 240
by 410
y_len 360
x_len 0
px 240
py 50
bx 240
by 400
y_len 350
x_len 0
px 240
py 50
bx 240
by 390
y_len 340
x_len 0
px 240
py 50
bx 240
by 380
y_len 330
x_len 0
px 240
py 50
bx 240
by 370
y_len 320
x_len 0
px 240
py 50
bx 240
by 360
y_len 310
x_len 0
px 240
py 50
bx 240
by 350
y_len 300
x_len 0
px 240
py 50
bx 240
by 340
y_len 290
x_len 0
px 240
py 50
bx 240
by 330
y_len 280
x_len 0
px 240
py 50
bx 240
by 320
y_len 270
x_len 0
px 240
py 50
bx 240
by 310
y_len 260
x_len 0
px 240
py 

px 241
py 24
bx 240
by 60
y_len 36
x_len 1
Move at angle
px 240
py 24
bx 240
by 60
y_len 36
x_len 0
Should be met...
px 239
py 24
bx 240
by 60
y_len 36
x_len 1
Move at angle
px 237
py 23
bx 240
by 60
y_len 37
x_len 3
Move at angle
px 236
py 22
bx 240
by 60
y_len 38
x_len 4
Move at angle
px 235
py 20
bx 240
by 60
y_len 40
x_len 5
Move at angle
px 234
py 19
bx 240
by 60
y_len 41
x_len 6
Move at angle
px 233
py 19
bx 240
by 60
y_len 41
x_len 7
Move at angle
px 233
py 18
bx 240
by 60
y_len 42
x_len 7
Move at angle
px 233
py 17
bx 240
by 60
y_len 43
x_len 7
Move at angle
px 232
py 16
bx 240
by 60
y_len 44
x_len 8
Move at angle
px 232
py 15
bx 240
by 60
y_len 45
x_len 8
Move at angle
px 231
py 14
bx 240
by 60
y_len 46
x_len 9
Move at angle
px 231
py 13
bx 240
by 60
y_len 47
x_len 9
Move at angle
px 232
py 13
bx 240
by 60
y_len 47
x_len 8
Move at angle
px 233
py 13
bx 240
by 60
y_len 47
x_len 7
Move at angle
px 235
py 12
bx 240
by 60
y_len 48
x_len 5
Move at angle
px 236
py 12
bx 240
by 60
y_

px 134
py 356
bx 240
by 30
y_len 326
x_len 106
Move at angle
px 139
py 361
bx 240
by 30
y_len 331
x_len 101
Move at angle
px 146
py 367
bx 240
by 30
y_len 337
x_len 94
Move at angle
px 152
py 372
bx 240
by 30
y_len 342
x_len 88
Move at angle
px 158
py 375
bx 240
by 30
y_len 345
x_len 82
Move at angle
px 163
py 378
bx 240
by 30
y_len 348
x_len 77
Move at angle
px 169
py 382
bx 240
by 30
y_len 352
x_len 71
Move at angle
px 176
py 385
bx 240
by 30
y_len 355
x_len 64
Move at angle
px 180
py 387
bx 240
by 30
y_len 357
x_len 60
Move at angle
px 184
py 389
bx 240
by 30
y_len 359
x_len 56
Move at angle
px 187
py 391
bx 240
by 30
y_len 361
x_len 53
Move at angle
px 190
py 393
bx 240
by 30
y_len 363
x_len 50
Move at angle
px 196
py 398
bx 240
by 30
y_len 368
x_len 44
Move at angle
px 207
py 406
bx 240
by 30
y_len 376
x_len 33
Move at angle
px 221
py 414
bx 240
by 30
y_len 384
x_len 19
Move at angle
px 239
py 423
bx 240
by 30
y_len 393
x_len 1
Move at angle
px 255
py 432
bx 240
by 30
y_len 402
x_

px 212
py 397
bx 240
by 30
y_len 367
x_len 28
Move at angle
px 234
py 397
bx 240
by 30
y_len 367
x_len 6
Move at angle
px 260
py 389
bx 240
by 30
y_len 359
x_len 20
Move at angle
px 283
py 380
bx 240
by 30
y_len 350
x_len 43
Move at angle
px 310
py 363
bx 240
by 30
y_len 333
x_len 70
Move at angle
px 335
py 345
bx 240
by 30
y_len 315
x_len 95
Move at angle
px 356
py 328
bx 240
by 30
y_len 298
x_len 116
Move at angle
px 373
py 313
bx 240
by 30
y_len 283
x_len 133
Move at angle
px 394
py 295
bx 240
by 30
y_len 265
x_len 154
Move at angle
px 411
py 280
bx 240
by 30
y_len 250
x_len 171
Move at angle
px 423
py 269
bx 240
by 30
y_len 239
x_len 183
Move at angle
px 433
py 260
bx 240
by 30
y_len 230
x_len 193
Move at angle
px 440
py 255
bx 240
by 30
y_len 225
x_len 200
Move at angle
px 447
py 250
bx 240
by 30
y_len 220
x_len 207
Move at angle
px 453
py 248
bx 240
by 30
y_len 218
x_len 213
Move at angle
px 457
py 247
bx 240
by 30
y_len 217
x_len 217
Move at angle
px 459
py 246
bx 240
by 30
y_le